# Large HW Swings

In [1]:
#Parameters
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET= 'TINYAES128C'
num_traces=50 #number of traces to capture

In [8]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes-independant.s aes_enc.s aes_keyschedule.s aes_sbox.s aes128_enc.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes-independant.d aes_enc.d aes_keyschedule.d aes_sbox.d aes128_enc.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes-independant.i aes_enc.i aes_keyschedule.i aes_sbox.i aes128_enc.i
.
-------- begin --------
avr-gcc (WinAVR 20100110) 4.3.3
Copyright (C) 2008 Free Software Foundation, Inc.
This is free software; see the source for copying conditio

## Data Tracing Theory

In the last tutorial, we saw how the power measurement of a device is related to the Hamming weight. Let's use this to see where some arbitrary data is processed by a device. We'll later expand on this to perform a test that also takes into account noise.

Our objective is simple - we'll send in some data with all 1's in one location, and then some data with all 0's.

## Capturing Power Traces

Capturing power traces will be very similar to previous tutorials, except this time we'll be using a loop to capture multiple traces, as well as numpy to store them.

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [14]:
%run "Helper_Scripts/Setup.ipynb"

In [15]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)

In [16]:
# program the target
scope.io.pdid = 'high_z'
cw.programTarget(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3471 bytes


### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and appends our new trace to the `traces[]` list. At the end, we convert the trace data to numpy arrays, since that's what we'll be using for analysis.

In [17]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

traces = []
textin = []
keys = []
target.init()
for i in tnrange(num_traces, desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    
    #Currently ALL bits are random. Let's extend bit 0 to a full byte to give us a random 0xFF or 0x00
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

Capturing traces: 100%|██████████| 100/100 [00:03<00:00, 28.22it/s]


Now that we have our traces, we can also plot them using Bokeh:

In [18]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(traces[0]))
p.line(xrange, traces[2], line_color="red")
show(p)

Loading BokehJS ...

In [19]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Trace Analysis

### Comparing 0xFF to 0x00

Now that we have some traces, let's look at what we've actually recorded. We'll be doing the following tasks:

1. Seperate traces into two groups: 0x00, and 0xFF
1. Make an average of each group.
1. Subtract the two averages and see the difference.

This will be shown in the following two cells. Note the number of 0xFF and 0x00 isn't exactly 50/50. That is why we need to ensure we average them.

In [20]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np

output_notebook()
p = figure()

one_list = []
zero_list = []

for tnum in range(0, len(traces)):
    if textin[tnum][0] == 0x00:
        one_list.append(traces[tnum])
    else:
        zero_list.append(traces[tnum])

print("Number of 0xFF: " + str(len(one_list)))
print("Number of 0x00: " + str(len(zero_list)))

Loading BokehJS ...

Number of 0xFF: 45
Number of 0x00: 55


In [21]:
one_avg = np.asarray(one_list).mean(axis=0)
zero_avg = np.asarray(zero_list).mean(axis=0)

diff = one_avg - zero_avg

p.line(range(0, len(traces[0])), diff)

show(p)

Notice the large spike as the data is handled. You could also try some of the following if you have time:

* Force the OUTPUT data to all 0xFFs or all 0x00s
* Force some intermediate value of 0xFFs or all 0x00s
* Set multiple bytes to 0xFF vs 0x00
* Plot each byte in sequence to see the data movement

## Tests

In [35]:
assert (max(abs(diff)) > 0.01), "Low max difference of {} between 0x00 and 0xFF".format(max(abs(diff)))